<a href="https://colab.research.google.com/github/AstroNoodles/Mini-Projects/blob/master/RNNTweetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network With Trump Generator Tweets in Keras

Rnns to use:


*   GRU
*   LSTM
*   AlphaDropout / Dropout - To apply to the layer





In [3]:
!pip install tensorflow

import tensorflow as tf
tf.enable_eager_execution()
# Import the libraries


import tensorflow.keras as keras
import pandas as pd
import numpy as np


In [0]:
csv_url = "https://raw.githubusercontent.com/lucamusk/Atomhacks2019TrumpBot/master/TrumpTweets.csv?token=AXOm-jThREnKNDIWCp2--ddMRzO1C2VWks5cqMiXwA%3D%3D"
df = pd.read_csv(csv_url, header=0)

tweets = df['text']


In [5]:
train_set_tweets = tweets[:1000]
train_set_tweets = train_set_tweets.str.replace(" &amp;", " &").str.ljust(280)

train_set_tweets.head(5)

0    A huge thanks to @HeatherNauert for serving Am...
1    So funny that The New York Times & The Washing...
2    ...The problem is no matter what the Radical L...
3    Robert Mueller was a Hero to the Radical Left ...
4    Had the Fed not mistakenly raised interest rat...
Name: text, dtype: object

In [0]:
concat_thousand_tweets = ""
for i in range(1000):
  concat_thousand_tweets += train_set_tweets[i]
  
vocab = sorted(set(concat_thousand_tweets))

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in train_set_tweets[0]]) 
test = np.array([char2idx[c] for c in train_set_tweets[1]])

for i in range(1, len(text_as_int)):
  text_as_int = np.concatenate((text_as_int, np.array([char2idx[c] for c 
                                                      in train_set_tweets[i]])))
                                                   


In [0]:
seq_length = 280
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [61]:
sequences = char_dataset.batch(seq_length, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))
  


RuntimeError: ignored

In [0]:
def split_input_target(chunk):
  
  total_chars = 0
  word_per_chunk = chunk.split(" ")
  for i in range(3):
    total_chars += len(word_per_chunk[i])
  total_chars += 3
  
  
  input_text = chunk[total_chars:]
  target_text = chunk[:total_chars]
  return input_text, target_text



In [0]:
dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024


In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size=64):
  model = keras.Sequential([
      keras.layers.Embedding(vocab_size, embedding_dim, 
                                batch_input_shape = [batch_size, None]),
      keras.layers.GRU(rnn_units, return_sequences=True,
          recurrent_initializer = 'glorot_uniform', recurrent_activation="sigmoid",stateful = True),
      keras.layers.Dense(vocab_size)
  ])
  return model

                

In [0]:
model = build_model(vocab_size = len(vocab),
                   embedding_dim = embedding_dim,
                   rnn_units = rnn_units,
                   batch_size = 280
                   )